In [2]:
# import section

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt 
from collections import Counter

In [21]:
users_df = pd.read_csv('C:/Users/yama/Documents/data projects unrelated to mypart/dh_yyr/data/train_users.csv', nrows=700000000000, low_memory=False)
accounts_df = pd.read_csv('C:/Users/yama/Documents/data projects unrelated to mypart/dh_yyr/data/train_accounts.csv', nrows=7000000000, low_memory=False)

In [22]:
accounts_df.head()

,account_id,account_name,created_at,plan_id,trial_start,churn_date,churn_reason,time_diff,region,country,...,payment_currency,max_team_size,min_team_size,industry,utm_cluster_id,mrr,user_goal,user_description,team_size,lead_score
0,1.0,"Gardner, Barron and Keller",2019-01-01 00:01:15,NaN,2019-01-01 00:01:15,NaN,NaN,11.0,New South Wales,AU,...,AUD,5.0,2.0,NaN,orders,NaN,NaN,NaN,NaN,0
1,2.0,Dunn Ltd,2019-01-01 00:01:52,NaN,2019-01-01 00:01:52,NaN,NaN,NaN,New Jersey,US,...,USD,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,3.0,Boone Inc,2019-01-01 00:03:12,NaN,2019-01-01 00:03:12,NaN,NaN,-6.0,Louisiana,US,...,USD,1.0,1.0,Other,todos,NaN,NaN,NaN,1,0
3,4.0,"Christian, Carroll and Davis",2019-01-01 00:04:11,NaN,2019-01-01 00:04:11,NaN,NaN,NaN,Tel Aviv,IL,...,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,5.0,Brooks-Oliver,2019-01-01 00:04:21,NaN,2019-04-04 11:09:12,NaN,NaN,-5.0,North Carolina,US,...,USD,1.0,1.0,Design,todos,NaN,NaN,NaN,1,0


In [23]:
index_of_useless_columns = accounts_df.nunique().isin([0,1])
columns_to_drop = np.array(accounts_df.columns)[index_of_useless_columns]
accounts_df.drop(columns=columns_to_drop, inplace=True)

In [24]:
accounts_df.head()

,account_id,account_name,created_at,plan_id,trial_start,churn_date,churn_reason,time_diff,region,country,...,payment_currency,max_team_size,min_team_size,industry,utm_cluster_id,mrr,user_goal,user_description,team_size,lead_score
0,1.0,"Gardner, Barron and Keller",2019-01-01 00:01:15,NaN,2019-01-01 00:01:15,NaN,NaN,11.0,New South Wales,AU,...,AUD,5.0,2.0,NaN,orders,NaN,NaN,NaN,NaN,0
1,2.0,Dunn Ltd,2019-01-01 00:01:52,NaN,2019-01-01 00:01:52,NaN,NaN,NaN,New Jersey,US,...,USD,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,3.0,Boone Inc,2019-01-01 00:03:12,NaN,2019-01-01 00:03:12,NaN,NaN,-6.0,Louisiana,US,...,USD,1.0,1.0,Other,todos,NaN,NaN,NaN,1,0
3,4.0,"Christian, Carroll and Davis",2019-01-01 00:04:11,NaN,2019-01-01 00:04:11,NaN,NaN,NaN,Tel Aviv,IL,...,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,5.0,Brooks-Oliver,2019-01-01 00:04:21,NaN,2019-04-04 11:09:12,NaN,NaN,-5.0,North Carolina,US,...,USD,1.0,1.0,Design,todos,NaN,NaN,NaN,1,0


In [25]:
time_between_created_trial = pd.to_datetime(accounts_df['trial_start']) - pd.to_datetime(accounts_df['created_at'])
accounts_df = accounts_df.assign(created_trial_delta=time_between_created_trial.apply(lambda x: (x.seconds//3600)))

In [26]:
accounts_df.head()

,account_id,account_name,created_at,plan_id,trial_start,churn_date,churn_reason,time_diff,region,country,...,max_team_size,min_team_size,industry,utm_cluster_id,mrr,user_goal,user_description,team_size,lead_score,created_trial_delta
0,1.0,"Gardner, Barron and Keller",2019-01-01 00:01:15,NaN,2019-01-01 00:01:15,NaN,NaN,11.0,New South Wales,AU,...,5.0,2.0,NaN,orders,NaN,NaN,NaN,NaN,0,0
1,2.0,Dunn Ltd,2019-01-01 00:01:52,NaN,2019-01-01 00:01:52,NaN,NaN,NaN,New Jersey,US,...,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,3.0,Boone Inc,2019-01-01 00:03:12,NaN,2019-01-01 00:03:12,NaN,NaN,-6.0,Louisiana,US,...,1.0,1.0,Other,todos,NaN,NaN,NaN,1,0,0
3,4.0,"Christian, Carroll and Davis",2019-01-01 00:04:11,NaN,2019-01-01 00:04:11,NaN,NaN,NaN,Tel Aviv,IL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,5.0,Brooks-Oliver,2019-01-01 00:04:21,NaN,2019-04-04 11:09:12,NaN,NaN,-5.0,North Carolina,US,...,1.0,1.0,Design,todos,NaN,NaN,NaN,1,0,11
